## Accuracy: 98.33% 

Things to note:
1. purchase cnt/Ancillary Category/Resident Base Discount different
2. "Sked Detail Id Nmbr" is null

In [1]:
from helpfile import *

# Import and clean snowflake views

In [2]:
query = """select * from ANALYTICS_PROD.IOATAWARE.VW_SALES_RAW_DATA where "Charge Date"='07/30/2023'
    """
snowflake_pd_df = get_data(query)

In [3]:
snowflake_pd_df = snowflake_pd_df[snowflake_pd_df[["Sked Detail Id Nmbr"]].notna().any(axis=1)]

In [4]:
columns_to_remove = ['\tPercent of Full Leg','Leg Status','Flight Time','Sales Username']
snowflake_pd_df = snowflake_pd_df.drop(columns=columns_to_remove)

snowflake_pd_df['Charge Date'] = pd.to_datetime(snowflake_pd_df['Charge Date'])
snowflake_pd_df['Flight Date'] = pd.to_datetime(snowflake_pd_df['Flight Date'])
snowflake_pd_df['Net Charge'] = snowflake_pd_df['Net Charge'].astype(float)
snowflake_pd_df['Taxes'] = snowflake_pd_df['Taxes'].astype(float)
snowflake_pd_df['Total Charge'] = snowflake_pd_df['Total Charge'].astype(float)
snowflake_pd_df['Flight Nmbr'] = snowflake_pd_df['Flight Nmbr'].astype(int)

In [5]:
#snowflake_pd_df.dtypes

# Import and clean report data

In [6]:
report_df = pd.read_csv('Sales/Sales_Raw_Data_0730.csv', skiprows=0)   

In [7]:
#'LastMod_SalesUser''PurchaseCnt','AncillaryCategory','Base_Taxes'
columns_to_remove = ['Source','mny_Tax_1_Percentage','ActualFlightDate','Percent','str_Leg_Status','LastMod_SalesUser','CancellationDate','PaxStatus', 'ChargeStatus', 'SeatSold']
report_df = report_df.drop(columns=columns_to_remove)

columns_to_process = ['NetCharge', 'Taxes','TotalCharge','Base_Charge','Base_Discount','Base_Taxes']

# Apply the function to specified columns in the dataframe
for col in columns_to_process:
    report_df[col] = report_df[col].apply(process_fee_column)

report_df['ChargeDate'] = pd.to_datetime(report_df['ChargeDate'])
report_df['ChargeDate'] =report_df['ChargeDate'].dt.strftime('%m/%d/%Y')
report_df['ChargeDate'] = pd.to_datetime(report_df['ChargeDate'])
report_df['FlightDate'] = pd.to_datetime(report_df['FlightDate'])
report_df['FlightDate'] =report_df['FlightDate'].dt.strftime('%m/%d/%Y')
report_df['FlightDate'] = pd.to_datetime(report_df['FlightDate'])

C:\Users\Ruiying.Zhang\AppData\Local\Temp\ipykernel_27836\2793499845.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  report_df['ChargeDate'] = pd.to_datetime(report_df['ChargeDate'])


In [8]:
#report_df.dtypes

# Compare two dataframes

In [10]:
report_accuracy,snowflake_accuracy,report_only,snowflake_only = calculate_accuracy(report_df,snowflake_pd_df)
print(f"Accuracy: {snowflake_accuracy:.2f}%" " snowflake records in report\nThe number of records in snowflake but not in report is", len(snowflake_only))
print(f"Accuracy: {report_accuracy:.2f}%"" report records in snowflake\nThe number of records in report but not in snowflake is", len(report_only))

Accuracy: 94.02% snowflake records in report
The number of records in snowflake but not in report is 3452
Accuracy: 94.06% report records in snowflake
The number of records in report but not in snowflake is 3429


# Output the difference file

In [13]:
try:
    snowflake_only.to_csv("Sales/"+"snowflake_only_record.csv", index=False) 
except:
    print("No snowflake only records")
try:
    report_only.to_csv("Sales/"+"report_only_record.csv", index=False) 
except:
    print("No report only records")

In [14]:
snow_res = list(snowflake_pd_df['Reservation Nmbr'].unique())
report_res = list(report_df['Reservation Nmbr'].unique())

In [15]:
elements_in_snow_not_in_report = list(set(snow_res) - set(report_res))
elements_in_report_not_in_snow = list(set(report_res)-set(snow_res))

snow_file_path = "Sales/snowonly_reservation.txt"
report_file_path = "Sales/reportonly_reservation.txt"

# Open the file for writing
with open(snow_file_path, "w") as file:
    for item in elements_in_snow_not_in_report:
        file.write("%s," % item)
with open(report_file_path, "w") as file:
    for item in elements_in_snow_not_in_report:
        file.write("%s," % item)